# Vorlesung Kollaborative Wissenschaft 

Simulation von Kooperationsverhalten

Gerd Graßhoff, Humboldt Universität zu Berlin

In [1]:
from dataclasses import dataclass, make_dataclass,field,asdict
from typing import List
import random
from random import sample
import pandas as pd

In [2]:
import qgrid

In [3]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [4]:
import plotly.express as px

# Epistemic systems
## Research results

In [5]:
@dataclass
class Vorhaben():
    id: int
    aufwand: float=100.0

In [6]:
# ein Forschungsobjekt des Vorhaben mit der ID=1
erg=Vorhaben(id=1)

In [7]:
print(f"Forschungsergebnis {erg} mit ID={erg.id}")

Forschungsergebnis Vorhaben(id=1, aufwand=100.0) mit ID=1


## List of results: knowledge

In [8]:
# Wissen
wissensumfang=5
wissen=[Vorhaben(k) for k in range(wissensumfang)]
wissen

[Vorhaben(id=0, aufwand=100.0),
 Vorhaben(id=1, aufwand=100.0),
 Vorhaben(id=2, aufwand=100.0),
 Vorhaben(id=3, aufwand=100.0),
 Vorhaben(id=4, aufwand=100.0)]

In [9]:
wissen[3].wert = 100
wissen[3].wert

100

In [10]:
# Zufallsauswahl von 3 Elementen
sample(wissen,3)

[Vorhaben(id=4, aufwand=100.0),
 Vorhaben(id=2, aufwand=100.0),
 Vorhaben(id=1, aufwand=100.0)]

In [11]:
def auswahl(wissen,key,val):
    c=[]
    for x in wissen:
        d=asdict(x)
        if d[key]==val:
            c.append(x)
    return(c)

In [12]:
auswahl(wissen,"aufwand",100)

[Vorhaben(id=0, aufwand=100.0),
 Vorhaben(id=1, aufwand=100.0),
 Vorhaben(id=2, aufwand=100.0),
 Vorhaben(id=3, aufwand=100.0),
 Vorhaben(id=4, aufwand=100.0)]

### Forscher

In [13]:
@dataclass
class Forscher:
    name: str
    energie: int
    last: str
    kompetenzen: List[Vorhaben]
        
    def __init__(self,name:str,kompetenzen=List[Vorhaben]):
                self.name=name
                self.energie=100
                self.kompetenzen=sample(wissen,2)

In [14]:
forscher=[Forscher("Bednorz"),Forscher("Müller")]
# Zahl Forscher
forscherzahl=len(forscher)
print(f"Zahl der Forscher = {forscherzahl}")
print(f"Forscher {forscher[1].name} hat die kompetenzen {forscher[1].kompetenzen}")

Zahl der Forscher = 2
Forscher Müller hat die kompetenzen [Vorhaben(id=1, aufwand=100.0), Vorhaben(id=0, aufwand=100.0)]


In [15]:
# Auswahl von zu erzielenden Ergebnissen für einen Forscher F
def forschungsziel(F):
    fz=[]
    for x in wissen:
        if x in F.kompetenzen:
            pass
        else:
            fz=x
            break
    return(fz)

In [16]:
def addKompetenz(F,x):
    '''Ein Forscher erwirbt das Ergebnis x als neue Komptenz'''
    # only add new competences x
    if x in F.kompetenzen:
        pass
    else:
        F.kompetenzen.append(x)

In [17]:
fz=forschungsziel(forscher[0])
print(f"forschungsziele des Forschers {forscher[0].name} sind: {fz}\n")
addKompetenz(forscher[0],fz)
print(f"Nach dem neuen Forschungsziel: **{fz}** verfügt der Forscher über neue Kompetenzen: {forscher[0].kompetenzen}")

forschungsziele des Forschers Bednorz sind: Vorhaben(id=0, aufwand=100.0)

Nach dem neuen Forschungsziel: **Vorhaben(id=0, aufwand=100.0)** verfügt der Forscher über neue Kompetenzen: [Vorhaben(id=1, aufwand=100.0), Vorhaben(id=4, aufwand=100.0), Vorhaben(id=0, aufwand=100.0)]


# Forschungshandlungen

In [18]:
def forscherWahl(F):
    for i in range(forscherzahl):
        NF=forscher[i]
        if F.name != NF.name:
            break
    return(NF)

In [19]:
def entscheidung(F,ziel):
    if F.name=="Kepler":
        dec="s"
    else:
        dec="t"
    return(dec)

In [20]:
matrix={"t":{"t":[7,7],"s":[0,10]},"s":{"t":[10,0],"s":[2,2]}}
matrix["t"]["s"]

[0, 10]

In [21]:
def forschungsschritt():
    A=forscher[0]
    E=forschungsziel(A)
    B=forscherWahl(A)
    decA=entscheidung(A,E) 
    decB=entscheidung(B,E)
    ergeb=matrix[decA][decB]
    if decA=="t" and decB=="t":
        addKompetenz(A,E)
        addKompetenz(B,E)
    if decA=="t" and decB=="s":
        addKompetenz(B,E)
    if decA=="s" and decB=="t":
        addKompetenz(A,E)
    if decA=="s" and decB=="s":
        pass
    A.last=decA
    B.last=decB
    A.energie=A.energie-ergeb[0]
    B.energie=B.energie-ergeb[1]

In [22]:
forscher[0].kompetenzen

[Vorhaben(id=1, aufwand=100.0),
 Vorhaben(id=4, aufwand=100.0),
 Vorhaben(id=0, aufwand=100.0)]

# Forschungsprozess
## Initialisierung

In [23]:
def forscherprofil():
    f0=forscher[0]
    f1=forscher[1]
    lW1=len(f0.kompetenzen)
    lW2=len(f1.kompetenzen)
    fp={"f1":f0.name,"w1":lW1,"en1":f0.energie,"last1":f0.last,"f2":f1.name,"w2":lW2,"en2":f1.energie,"last2":f1.last}
    return(fp)

In [24]:
## Dataframe als Tagebuch
forschungsProzess=pd.DataFrame(columns=["f1","w1","en1","last1","f2","w2","en2","last2"])
## forscher
forscher=[Forscher("Kepler")]
forscher.append(Forscher("Brahe"))
## wissen
umfangwissen=50
wissen=[Vorhaben(k) for k in range(umfangwissen)]

## Simulation

In [25]:
for s in range(10):
    forschungsschritt()
    forschungsProzess=forschungsProzess.append(forscherprofil(),ignore_index=True)

In [26]:
forschungsProzess.head(10)

,f1,w1,en1,last1,f2,w2,en2,last2
0,Kepler,3,90,s,Brahe,2,100,t
1,Kepler,4,80,s,Brahe,2,100,t
2,Kepler,5,70,s,Brahe,2,100,t
3,Kepler,6,60,s,Brahe,2,100,t
4,Kepler,7,50,s,Brahe,2,100,t
5,Kepler,8,40,s,Brahe,2,100,t
6,Kepler,9,30,s,Brahe,2,100,t
7,Kepler,10,20,s,Brahe,2,100,t
8,Kepler,11,10,s,Brahe,2,100,t
9,Kepler,12,0,s,Brahe,2,100,t


# Dashboard Simulation Forschungskollaboration

In [30]:
@interact
def show_articles_more_than(column='en1', x=100):
    return forschungsProzess.loc[forschungsProzess[column] < x]

interactive(children=(Text(value='en1', description='column'), IntSlider(value=100, description='x', max=300, …

In [28]:
forschungsProzess["w1"].iplot(kind='hist', xTitle='Zahl Kollaborationen',
                  yTitle='count', title='Kompetenzentwicklung')

AttributeError: 'Series' object has no attribute 'iplot'

# Visualisation

https://plot.ly/python/plotly-express/

https://towardsdatascience.com/the-next-level-of-data-visualization-in-python-dd6e99039d5e

In [29]:
fig=ox.bar(forschungsProzess,x=)

SyntaxError: invalid syntax (<ipython-input-29-f41d3fcd182a>, line 1)